In [1]:
# load libraies

%run py_libraries.py

/Users/4476224/.local/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.6.0 and strictly below 2.9.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.13.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# loading utility files

from utility.sv_fig import savefig
from utility.make_cm import make_confusion_matrix
from utility.get_g_result import get_gamma_results
from utility.plt_result import plot_results

In [3]:
def savefig(filename, crop = True):
    plt.savefig('{}.pdf'.format(filename))

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_row', None)

# load data
X_train = pd.read_csv('data/norm_X_train_PCa_Ca.csv')
X_test = pd.read_csv('data/norm_X_test_PCa_Ca.csv')
y_train = pd.read_csv('data/y_train_PCa_Ca.csv')
y_test = pd.read_csv('data/y_test_PCa_Ca.csv')


print(X_train.shape)

(109, 37)


In [5]:
# Print the column names as a list
print(X_train.columns.tolist())

['ENA.78', 'IFN.y', 'IL.10', 'IL.6', 'IL.8', 'MCP.1', 'MDC', 'MIP.1a', 'TNF.a', 'C.peptide', 'G.CSF', 'IL.22', 'Insulin', 'Leptin', 'MIP.3a', 'GRO.a', 'HGF', 'MMP.2', 'Adiponectin', 'CRP', 'GDF.15', 'TIMP.1', 'TGF.B2', 'TGF.B1', 'PPAR.y', 'HIF.1a', 'Laminin', 'HbA1c', 'CA19.9', 'Glucose', 'HDL', 'CCK', 'LDL', 'Triglyceride', 'Albumin', 'Lumican', 'ZAG']


In [6]:
Xy_PCa_Ca = pd.concat([X_train, y_train], axis=1)

In [7]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from scipy import stats

col_to_scale = ['ENA.78', 'IFN.y', 'IL.10', 'IL.6', 'IL.8', 'MCP.1', 'MDC', 'MIP.1a', 'TNF.a', 'C.peptide', 'G.CSF', 'IL.22', 'Insulin', 'Leptin', 
                'MIP.3a', 'GRO.a', 'HGF', 'MMP.2', 'Adiponectin', 'CRP', 'GDF.15', 'TIMP.1', 'TGF.B2', 'TGF.B1', 'PPAR.y', 'HIF.1a', 'Laminin', 
                'HbA1c', 'CA19.9', 'Glucose', 'HDL', 'CCK', 'LDL', 'Triglyceride', 'Albumin', 'Lumican', 'ZAG', ]

def midas_impl(missing, col_to_scale, n_epoch):
    missing_cp = missing.copy() # make copy of missing df
    
    # scaler = MinMaxScaler().fit(missing_cp[col_to_scale]) # scale data
    scaler = StandardScaler().fit(missing_cp[col_to_scale]) # scale data
    missing_sc = scaler.transform(missing_cp[col_to_scale]) # this is an array, convert to df

    missing_df = pd.DataFrame({'ENA.78':missing_sc[:,0],
                               'IFN.y':missing_sc[:,1],
                               'IL.10':missing_sc[:,2],
                               'IL.6':missing_sc[:,3],
                               'IL.8':missing_sc[:,4],
                               'MCP.1':missing_sc[:,5],
                               'MDC':missing_sc[:,6],
                               'MIP.1a':missing_sc[:,7],
                               'TNF.a':missing_sc[:,8],
                               'C.peptide':missing_sc[:,9],
                               'G.CSF':missing_sc[:,10],
                               'IL.22':missing_sc[:,11],
                               'Insulin':missing_sc[:,12],
                               'Leptin':missing_sc[:,13],
                               'MIP.3a':missing_sc[:,14],
                               'GRO.a':missing_sc[:,15],
                               'HGF':missing_sc[:,16],
                               'MMP.2':missing_sc[:,17],
                               'Adiponectin':missing_sc[:,18],
                               'CRP':missing_sc[:,19],
                               'GDF.15':missing_sc[:,20],
                               'TIMP.1':missing_sc[:,21],
                               'TGF.B2':missing_sc[:,22],
                               'TGF.B1':missing_sc[:,23],
                               'PPAR.y':missing_sc[:,24],
                               'HIF.1a':missing_sc[:,25],
                               'Laminin':missing_sc[:,26],
                               'HbA1c':missing_sc[:,27],
                               'CA19.9':missing_sc[:,28],
                               'Glucose':missing_sc[:,29],
                               'HDL':missing_sc[:,30],
                               'CCK':missing_sc[:,31],
                               'LDL':missing_sc[:,32],
                               'Triglyceride':missing_sc[:,33],
                               'Albumin':missing_sc[:,34],
                               'Lumican':missing_sc[:,35],
                               'ZAG':missing_sc[:,36],
                               })
    
    # Extract cat columns from missing
    ext_col1 = missing_cp['CACHEXSTAGE0VIG']
    

    # Add the extracted cat columns to missing_df
    missing_df = pd.concat([missing_df, ext_col1.rename('CACHEXSTAGE0VIG')], axis=1)
    

    missing_df.to_csv('missing_PCa_Ca.csv')
    data_0 = pd.read_csv('missing_PCa_Ca.csv')
    data_0.columns.str.strip()

    categorical = ['CACHEXSTAGE0VIG']
    
    data_cat, cat_cols_list = md.cat_conv(data_0[categorical])

    data_0.drop(categorical, axis = 1, inplace = True)
    constructor_list = [data_0]
    constructor_list.append(data_cat)
    data_in = pd.concat(constructor_list, axis=1)

    na_loc = data_in.isnull()
    data_in[na_loc] = np.nan

    # imputer = md.Midas(layer_structure = [128,128], vae_layer = True, seed = 89, input_drop = 0.75, learn_rate = 0.0001)
    imputer = md.Midas(layer_structure = [128,128], vae_layer = False, seed = 42, input_drop = 0.75, learn_rate = 0.001)
    imputer.build_model(data_in, softmax_columns = cat_cols_list)
    imputer.train_model(training_epochs = n_epoch)

    # do not update M
    imputations = imputer.generate_samples(m=20).output_list

    flat_cats = [cat for variable in cat_cols_list for cat in variable]
    for i in range(len(imputations)):
        tmp_cat = [imputations[i][x].idxmax(axis=1) for x in cat_cols_list]
        cat_df = pd.DataFrame({categorical[i]:tmp_cat[i] for i in range(len(categorical))})
        imputations[i] = pd.concat([imputations[i], cat_df], axis = 1).drop(flat_cats, axis = 1)

    mapping_ca = {'CACHEXSTAGE0VIG_0.0': 0,'CACHEXSTAGE0VIG_1.0': 1}

    imputations[0] = imputations[0].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[1] = imputations[1].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[2] = imputations[2].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[3] = imputations[3].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[4] = imputations[4].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[5] = imputations[5].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[6] = imputations[6].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[7] = imputations[7].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[8] = imputations[8].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[9] = imputations[9].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[10] = imputations[10].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[11] = imputations[11].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[12] = imputations[12].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[13] = imputations[13].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[14] = imputations[14].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[15] = imputations[15].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[16] = imputations[16].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[17] = imputations[17].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[18] = imputations[18].replace({'CACHEXSTAGE0VIG': mapping_ca})
    imputations[19] = imputations[19].replace({'CACHEXSTAGE0VIG': mapping_ca})

    # rescale imputations
    imputations[0][col_to_scale] = scaler.inverse_transform(imputations[0][col_to_scale])
    imputations[1][col_to_scale] = scaler.inverse_transform(imputations[1][col_to_scale])
    imputations[2][col_to_scale] = scaler.inverse_transform(imputations[2][col_to_scale])
    imputations[3][col_to_scale] = scaler.inverse_transform(imputations[3][col_to_scale])
    imputations[4][col_to_scale] = scaler.inverse_transform(imputations[4][col_to_scale])
    imputations[5][col_to_scale] = scaler.inverse_transform(imputations[5][col_to_scale])
    imputations[6][col_to_scale] = scaler.inverse_transform(imputations[6][col_to_scale])
    imputations[7][col_to_scale] = scaler.inverse_transform(imputations[7][col_to_scale])
    imputations[8][col_to_scale] = scaler.inverse_transform(imputations[8][col_to_scale])
    imputations[9][col_to_scale] = scaler.inverse_transform(imputations[9][col_to_scale])
    imputations[10][col_to_scale] = scaler.inverse_transform(imputations[10][col_to_scale])
    imputations[11][col_to_scale] = scaler.inverse_transform(imputations[11][col_to_scale])
    imputations[12][col_to_scale] = scaler.inverse_transform(imputations[12][col_to_scale])
    imputations[13][col_to_scale] = scaler.inverse_transform(imputations[13][col_to_scale])
    imputations[14][col_to_scale] = scaler.inverse_transform(imputations[14][col_to_scale])
    imputations[15][col_to_scale] = scaler.inverse_transform(imputations[15][col_to_scale])
    imputations[16][col_to_scale] = scaler.inverse_transform(imputations[16][col_to_scale])
    imputations[17][col_to_scale] = scaler.inverse_transform(imputations[17][col_to_scale])
    imputations[18][col_to_scale] = scaler.inverse_transform(imputations[18][col_to_scale])
    imputations[19][col_to_scale] = scaler.inverse_transform(imputations[19][col_to_scale])

    n=0
    for i in imputations:
        file_out_csv = "midas_PCa_Ca/midas_PCa_Ca_" + str(n) + ".csv"
        i.to_csv(file_out_csv, index=False)
        n += 1

    df0 = pd.read_csv('midas_PCa_Ca/midas_PCa_Ca_0.csv'); df0 = df0.iloc[:,1:]
    df1 = pd.read_csv('midas_PCa_Ca/midas_PCa_Ca_1.csv'); df1 = df1.iloc[:,1:]
    df2 = pd.read_csv('midas_PCa_Ca/midas_PCa_Ca_2.csv'); df2 = df2.iloc[:,1:]
    df3 = pd.read_csv('midas_PCa_Ca/midas_PCa_Ca_3.csv'); df3 = df3.iloc[:,1:]
    df4 = pd.read_csv('midas_PCa_Ca/midas_PCa_Ca_4.csv'); df4 = df4.iloc[:,1:]
    df5 = pd.read_csv('midas_PCa_Ca/midas_PCa_Ca_5.csv'); df5 = df5.iloc[:,1:]
    df6 = pd.read_csv('midas_PCa_Ca/midas_PCa_Ca_6.csv'); df6 = df6.iloc[:,1:]
    df7 = pd.read_csv('midas_PCa_Ca/midas_PCa_Ca_7.csv'); df7 = df7.iloc[:,1:]
    df8 = pd.read_csv('midas_PCa_Ca/midas_PCa_Ca_8.csv'); df8 = df8.iloc[:,1:]
    df9 = pd.read_csv('midas_PCa_Ca/midas_PCa_Ca_9.csv'); df9 = df9.iloc[:,1:]
    df10 = pd.read_csv('midas_PCa_Ca/midas_PCa_Ca_10.csv'); df10 = df10.iloc[:,1:]
    df11 = pd.read_csv('midas_PCa_Ca/midas_PCa_Ca_11.csv'); df11 = df11.iloc[:,1:]
    df12 = pd.read_csv('midas_PCa_Ca/midas_PCa_Ca_12.csv'); df12 = df12.iloc[:,1:]
    df13 = pd.read_csv('midas_PCa_Ca/midas_PCa_Ca_13.csv'); df13 = df13.iloc[:,1:]
    df14 = pd.read_csv('midas_PCa_Ca/midas_PCa_Ca_14.csv'); df14 = df14.iloc[:,1:]
    df15 = pd.read_csv('midas_PCa_Ca/midas_PCa_Ca_15.csv'); df15 = df15.iloc[:,1:]
    df16 = pd.read_csv('midas_PCa_Ca/midas_PCa_Ca_16.csv'); df16 = df16.iloc[:,1:]
    df17 = pd.read_csv('midas_PCa_Ca/midas_PCa_Ca_17.csv'); df17 = df17.iloc[:,1:]
    df18 = pd.read_csv('midas_PCa_Ca/midas_PCa_Ca_18.csv'); df18 = df18.iloc[:,1:]
    df19 = pd.read_csv('midas_PCa_Ca/midas_PCa_Ca_19.csv'); df19 = df19.iloc[:,1:]

    # Initialize the output dataframe with the same dimensions
    output_df = pd.DataFrame(index=df0.index, columns=df0.columns)

    # Process continuous columns
    for col in df1.columns[:35]:
        output_df[col] = (abs(df0[col]) + abs(df1[col]) + abs(df2[col]) + abs(df3[col]) + abs(df4[col]) + 
                          abs(df5[col]) + abs(df6[col]) + abs(df7[col]) + abs(df8[col]) + abs(df9[col]) + 
                         abs(df10[col]) + abs(df11[col]) + abs(df12[col]) + abs(df13[col]) + abs(df14[col]) + 
                          abs(df15[col]) + abs(df16[col]) + abs(df17[col]) + abs(df18[col]) + abs(df19[col])) / 20

    # Process categorical columns
    for col in df1.columns[35:]:
        output_df[col] = pd.concat([df0[col],df1[col],df2[col],df3[col],df4[col],df5[col],df6[col],df7[col],df8[col],df9[col],
                                   df10[col],df11[col],df12[col],df13[col],df14[col],df15[col],df16[col],df17[col],df18[col],df19[col]], axis=1).median(axis=1)
        
        # median = stats.median([df0[col],df1[col],df2[col],df3[col],df4[col],df5[col],df6[col],df7[col],df8[col],df9[col],
        #                   df10[col],df11[col],df12[col],df13[col],df14[col],df15[col],df16[col],df17[col],df18[col],df19[col]], axis=0).median[0]
        # output_df[col] = median

    # Write the output dataframe to a CSV file
    output_df.to_csv('output_PCa_Ca.csv', index=False)

    # re-order cols in output_df
    output_df_re = output_df[['ENA.78', 'IFN.y', 'IL.10', 'IL.6', 'IL.8', 'MCP.1', 'MDC', 'MIP.1a', 'TNF.a', 'C.peptide', 'G.CSF', 'IL.22', 'Insulin', 'Leptin', 
                'MIP.3a', 'GRO.a', 'HGF', 'MMP.2', 'Adiponectin', 'CRP', 'GDF.15', 'TIMP.1', 'TGF.B2', 'TGF.B1', 'PPAR.y', 'HIF.1a', 'Laminin', 
                'HbA1c', 'CA19.9', 'Glucose', 'HDL', 'CCK', 'LDL', 'Triglyceride', 'Albumin', 'Lumican', 'ZAG', 'CACHEXSTAGE0VIG']].copy()

    return output_df_re





In [8]:
# calculate proportion of missingness in ndata_noR

# total num of NaN in the ndata_noR
total_nan_count = X_train.isna().sum().sum()

# total num of cells in ndata_noR
total_cells = X_train.size

# proportion of NaN
NaN_proportion = total_nan_count / total_cells

print('NaN_proportions:',NaN_proportion)


NaN_proportions: 0.01909248698239524


In [9]:
# 1.35% missingness

Xy_PCa_Ca_md = midas_impl(Xy_PCa_Ca, col_to_scale, 200)

Size index: [38, 2]

Computation graph constructed

Metal device set to: Apple M1 Pro
Model initialised

Epoch: 0 , loss: 119.39386367797852
Epoch: 1 , loss: 101.97204462687175
Epoch: 2 , loss: 96.7711181640625
Epoch: 3 , loss: 93.47258122762044
Epoch: 4 , loss: 90.07582473754883
Epoch: 5 , loss: 90.192933400472
Epoch: 6 , loss: 80.53006490071614
Epoch: 7 , loss: 80.37185541788737
Epoch: 8 , loss: 74.59093348185222
Epoch: 9 , loss: 69.55303128560384
Epoch: 10 , loss: 72.39547983805339
Epoch: 11 , loss: 69.19249725341797
Epoch: 12 , loss: 63.826948165893555
Epoch: 13 , loss: 72.40936024983723
Epoch: 14 , loss: 66.71376164754231
Epoch: 15 , loss: 65.16867192586263
Epoch: 16 , loss: 61.19698905944824
Epoch: 17 , loss: 66.70373598734538
Epoch: 18 , loss: 69.50631395975749
Epoch: 19 , loss: 63.2539119720459
Epoch: 20 , loss: 66.77536010742188
Epoch: 21 , loss: 68.00705782572429
Epoch: 22 , loss: 62.28950627644857
Epoch: 23 , loss: 61.72060012817383
Epoch: 24 , loss: 64.47215016682942
Epoch:

In [10]:
Xy_PCa_Ca_md

,ENA.78,IFN.y,IL.10,IL.6,IL.8,MCP.1,MDC,MIP.1a,TNF.a,C.peptide,G.CSF,IL.22,Insulin,Leptin,MIP.3a,GRO.a,HGF,MMP.2,Adiponectin,CRP,GDF.15,TIMP.1,TGF.B2,TGF.B1,PPAR.y,HIF.1a,Laminin,HbA1c,CA19.9,Glucose,HDL,CCK,LDL,Triglyceride,Albumin,Lumican,ZAG,CACHEXSTAGE0VIG
0,0.744650,0.557853,0.446012,0.714546,0.655580,0.861103,0.843886,0.571410,0.710722,0.884702,0.666531,0.016406,0.707991,0.728342,0.433491,0.777142,0.671053,0.933395,0.888760,0.868732,0.740052,0.849915,0.310790,0.933222,0.090175,0.626646,0.815439,0.667915,0.262062,0.777176,0.862105,0.823278,0.820658,0.588362,0.983717,0.913480,0.934374,0.0
1,0.927977,0.732807,0.126919,0.319096,0.647052,0.896732,0.859788,0.492834,0.582937,0.932089,0.574505,1.000000,0.868005,0.817211,0.648108,0.823025,0.673495,0.760344,0.952149,0.775107,0.779672,0.917617,0.698572,0.938235,0.711507,0.678757,0.862666,0.937233,0.733067,0.782834,0.749988,0.849245,0.824813,0.836250,0.984218,0.905137,0.949461,1.0
2,0.850448,0.657211,0.788123,0.568915,0.963122,0.847864,0.797817,0.647590,0.921340,0.812709,0.536401,0.341878,0.447883,0.740362,0.631043,0.839095,0.621392,0.878573,0.933096,0.782424,0.879261,0.925266,0.631576,0.803223,0.523157,0.718528,0.970682,0.796330,0.955222,0.917829,0.914987,0.646777,0.857697,0.843750,0.986123,0.877855,0.820546,0.0
3,0.863223,0.657475,0.584844,0.541543,0.737232,0.910148,0.843613,0.565234,0.730560,0.765593,0.706727,0.418134,0.548933,0.786432,0.536219,0.832451,0.607072,0.938210,0.914765,0.857753,0.811587,0.858382,0.559844,0.908812,0.042141,0.678991,0.751398,0.829083,0.605416,0.846539,0.828993,0.819294,0.871974,0.575765,0.981555,0.963302,0.965647,0.0
4,0.813307,0.692730,0.426482,0.546201,0.861385,0.869692,0.899981,0.536252,0.684849,0.819017,0.593048,0.147066,0.572142,0.645286,0.510086,0.811799,0.665667,0.964735,0.969817,0.904992,0.921549,0.920082,0.829242,0.949066,0.296636,0.864750,0.911904,0.881816,0.162216,0.784777,0.886145,0.800947,0.831997,0.785852,0.984722,0.899271,0.888329,0.0
5,0.906643,0.551507,0.424801,0.474868,0.665381,0.861785,0.827269,0.494341,0.691641,0.901461,0.568898,0.082392,0.855303,0.787884,0.547048,0.879372,0.626824,0.894881,0.916756,0.872187,0.694660,0.795240,0.797926,0.965122,0.072824,0.662341,0.808608,0.862826,0.996085,0.834134,0.869338,0.656566,0.782205,0.860407,0.979587,0.948582,0.981832,1.0
6,0.941292,0.797714,0.516300,0.586412,0.679520,0.888082,0.878561,0.623870,0.782468,0.860626,0.626995,0.300633,0.691520,0.765775,0.512402,0.932921,0.687565,0.916178,0.899538,0.645915,0.808791,0.852678,0.689296,0.946620,0.274502,0.697791,0.923171,0.892142,0.096606,0.867775,0.806869,1.000000,0.913086,0.615529,0.960147,0.973904,0.982832,1.0
7,0.909215,0.689237,0.412986,0.512607,0.641187,0.840712,0.814715,0.622581,0.688696,0.790345,0.578632,0.158690,0.524362,0.769221,0.516095,0.943211,0.629969,0.950150,0.947312,0.718836,0.853585,0.864326,0.732558,0.890641,0.375882,0.608540,0.858427,0.788684,0.157760,0.817707,0.859636,0.915074,0.862865,0.732082,0.991299,0.972028,0.962750,0.0
8,0.782120,0.416794,0.088175,0.308315,0.920029,0.868122,0.832010,0.496698,0.546357,0.853743,0.625815,0.205720,0.680385,0.730190,0.313915,0.779499,0.607127,0.909759,0.949616,0.660459,0.773223,0.856900,0.721598,0.905861,0.605079,0.802964,0.872674,0.737239,0.844796,0.962675,0.728526,0.760191,0.783675,0.548274,0.991092,0.888217,0.903841,1.0
9,0.922472,0.725236,0.266401,0.423862,0.604761,0.895962,0.870430,0.454522,0.602578,0.829473,0.674292,0.185518,0.626880,0.793832,0.346388,0.817735,0.628717,0.729572,0.934284,0.822957,0.812189,0.878224,0.871266,0.940853,0.144221,0.892677,0.852676,0.866967,0.734586,0.816189,0.890765,0.860487,0.806114,0.694466,0.985589,0.971913,0.992271,1.0


In [11]:
# save to csv file

# 
Xy_PCa_Ca_md.to_csv('data/Xy_PCa_Ca_md.csv', index=False)